In [19]:
import numpy as np
import pandas as pd
import datetime
import math
from dateutil import parser
from datetime import datetime, timedelta
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  
from sklearn.metrics import classification_report, confusion_matrix  

# accessing files / converting timestamps

In [20]:

translationdf = pd.read_csv('data/in-motion.csv')
stilldf = pd.read_csv('data/stationary.csv')
swipedf = pd.read_csv('data/swipe.csv')
rotatedf = pd.read_csv('data/rotating.csv')
coveredf = pd.read_csv('data/cover.csv')
horizontaldf = pd.read_csv('data/horizontal.csv')


# stilldf['timestamp'] = (pd.Series(stilldf['LastSeenTimestampUTC'])) / 1000000  #converting microseconds to seconds
# start = datetime(1970, 1, 1)  # Unix epoch start time
# stilldf['datetime'] = stilldf.timestamp.apply(lambda x: start + timedelta(seconds=x)) #converted epoch microseconds to

def convert_date_time(df):
    df['timestamp'] = (pd.Series(df['LastSeenTimestampUTC'])) / 1000000  #converting microseconds to seconds
    start = datetime(1970, 1, 1)  # Unix epoch start time
    df['datetime'] = df.timestamp.apply(lambda x: start + timedelta(seconds=x)) #converted epoch microseconds to
    
convert_date_time(stilldf)  
convert_date_time(translationdf)
convert_date_time(swipedf)
convert_date_time(rotatedf)
convert_date_time(coveredf)
convert_date_time(horizontaldf)



# features 1 - 3

In [21]:
def compute_std(array): #feature 1
    if np.std(array) == 0:
        return None
    else:
        return np.std(array)
    
    

def compute_mean_difference(array): #feature3
    if len(array) == 0 or len(array) == 1:
        return None
    else:
        return np.mean(np.diff(array))
    

# decide which one is better sample or population
def compute_mean_rssi_std(array):
    freq = {}
    for x in array:
        if x[1] not in freq:
            freq[x[1]] = [x[0]]
        else:
            freq[x[1]].append(x[0])
    totalstds = []
    for x,y in freq.items():
        totalstds.append(np.std(y))
    if len(totalstds) != 0:
        if np.mean(totalstds) == 0:
            return None
        else:
            return np.mean(totalstds)
    else:
        return None
    


    
    
# for stationary 
stillchannel = [i for i in stilldf['ChannelIndex']]
stillrssi = [i for i in stilldf['PeakRSSI']]
stilltuplefeature2 = list(zip(stillrssi,stillchannel)) # to calculate feature 2

feature2stillseries = pd.Series(stilltuplefeature2, index=stilldf['datetime'])
stillseries = pd.Series(stillrssi, index=stilldf['datetime'])

stillfeature1 = pd.Series(stillseries.resample('2s').apply(compute_std))  #calculating feature 1
stillfeature2 = pd.Series(feature2stillseries.resample('2s').apply(compute_mean_rssi_std)) #calculating feature 2
stillfeature3 = pd.Series(stillseries.resample('2s').apply(compute_mean_difference)) #calculating feature 3


# for in motion
# ******************************************************************

translationchannel = [i for i in translationdf['ChannelIndex']]
translationrssi = [i for i in translationdf['ImpinjPeakRSSI']]


translationtuplefeature2 = list(zip(translationrssi,translationchannel)) # to calculate feature 2

feature2translationseries = pd.Series(translationtuplefeature2, index=translationdf['datetime'])
translationseries = pd.Series(translationrssi, index=translationdf['datetime'])

translationfeature1 = pd.Series(translationseries.resample('2s').apply(compute_std))  #calculating feature 1
translationfeature2 = pd.Series(feature2translationseries.resample('2s').apply(compute_mean_rssi_std)) #calculating feature 2
translationfeature3 = pd.Series(translationseries.resample('2s').apply(compute_mean_difference)) #calculating feature 3

# for swipe
# ******************************************************************

swipechannel = [i for i in swipedf['ChannelIndex']]
swiperssi = [i for i in swipedf['ImpinjPeakRSSI']]


swipetuplefeature2 = list(zip(swiperssi,swipechannel)) # to calculate feature 2

feature2swipeseries = pd.Series(swipetuplefeature2, index=swipedf['datetime'])
swipeseries = pd.Series(swiperssi, index=swipedf['datetime'])

swipefeature1 = pd.Series(swipeseries.resample('2s').apply(compute_std))  #calculating feature 1
swipefeature2 = pd.Series(feature2swipeseries.resample('2s').apply(compute_mean_rssi_std)) #calculating feature 2
swipefeature3 = pd.Series(swipeseries.resample('2s').apply(compute_mean_difference)) #calculating feature 3

#for rotate DF
# ******************************************************************

rotatechannel = [i for i in rotatedf['ChannelIndex']]
rotaterssi = [i for i in rotatedf['ImpinjPeakRSSI']]


rotatetuplefeature2 = list(zip(rotaterssi,rotatechannel)) # to calculate feature 2

feature2rotateseries = pd.Series(rotatetuplefeature2, index=rotatedf['datetime'])
rotateseries = pd.Series(rotaterssi, index=rotatedf['datetime'])

rotatefeature1 = pd.Series(rotateseries.resample('2s').apply(compute_std))  #calculating feature 1
rotatefeature2 = pd.Series(feature2rotateseries.resample('2s').apply(compute_mean_rssi_std)) #calculating feature 2
rotatefeature3 = pd.Series(rotateseries.resample('2s').apply(compute_mean_difference)) #calculating feature 3

#for cover DF
# ******************************************************************

coverchannel = [i for i in coveredf['ChannelIndex']]
coverrssi = [i for i in coveredf['ImpinjPeakRSSI']]


covertuplefeature2 = list(zip(coverrssi,coverchannel)) # to calculate feature 2

feature2coverseries = pd.Series(covertuplefeature2, index=coveredf['datetime'])
coverseries = pd.Series(coverrssi, index=coveredf['datetime'])

coverfeature1 = pd.Series(coverseries.resample('2s').apply(compute_std))  #calculating feature 1
coverfeature2 = pd.Series(feature2coverseries.resample('2s').apply(compute_mean_rssi_std)) #calculating feature 2
coverfeature3 = pd.Series(coverseries.resample('2s').apply(compute_mean_difference)) #calculating feature 3

#for horizontal DF
# ******************************************************************

horizontalchannel = [i for i in horizontaldf['ChannelIndex']]
horizontalrssi = [i for i in horizontaldf['ImpinjPeakRSSI']]


horizontaltuplefeature2 = list(zip(horizontalrssi,horizontalchannel)) # to calculate feature 2

feature2horizontalseries = pd.Series(horizontaltuplefeature2, index=horizontaldf['datetime'])
horizontalseries = pd.Series(horizontalrssi, index=horizontaldf['datetime'])

horizontalfeature1 = pd.Series(horizontalseries.resample('2s').apply(compute_std))  #calculating feature 1
horizontalfeature2 = pd.Series(feature2horizontalseries.resample('2s').apply(compute_mean_rssi_std)) #calculating feature 2
horizontalfeature3 = pd.Series(horizontalseries.resample('2s').apply(compute_mean_difference)) #calculating feature 3




# Feature 8 (Read Rate)

In [ ]:
#compute Feature 8

def read_rate(array): #Read Rate-Numberof packets receivedfromeachRFIDtag persecond
    if len(array) == 0.0:
        return None
    else:
        return len(array)


def pre_process_epc(df):
    epc = [i for i in df['EPC-96']]
    series = pd.Series(epc, index=df['datetime'])
    feature8 = pd.Series(series.resample('1s').apply(read_rate)) #calculating feature 8
    return feature8
    
    
stillfeature8 = pre_process_epc(stilldf)
translationfeature8 = pre_process_epc(translationdf)

# print(translationfeature8)

# Features 4 - 7

In [ ]:
#compute features 4 - 6 
#still need to address channel index


def c_f_p_r(array): #CFPR - Constant Frequency Phase Rate
    if len(array) == 0 or len(array) == 1:
        return None
    else:
        cfprs = []
        vfprs = []
        for i in range(len(array) - 1):
            if array[i][1] == array[i+1][1]:
                cfprs.append(array[i+1][0] - array[i][0]) # CFPR
            if array[i][1] != array[i+1][1]:
                vfprs.append((array[i+1][0] - array[i][0]) / (array[i+1][1] - array[i][1])) # VFPR
                
                
        featureTuple = [np.median(cfprs),sum(map(abs,cfprs)),np.std(cfprs,ddof=1),np.std(vfprs,ddof=1)]
        print(featureTuple,len(array))
        return featureTuple
                
    

# computing feature extraction for still
stillphase = [i for i in stilldf['ImpinjPhase']]
stillchannel = [i for i in stilldf['ChannelIndex']]
stilltuple = list(zip(stillphase,stillchannel))
    
stillseries = pd.Series(stilltuple, index=stilldf['datetime']) #tuple is formatted(phase,channel)

    
stillfeature4567 = pd.Series(stillseries.resample('2s').apply(c_f_p_r))  #calculating feature 4 5 6 7

                
# computing feature extraction for translation
translationphase = [i for i in translationdf['ImpinjPhase']]
translationchannel = [i for i in translationdf['ChannelIndex']]
translationtuple = list(zip(translationphase,translationchannel))
    
translationseries = pd.Series(translationtuple, index=translationdf['datetime']) #tuple is formatted(phase,channel)

    
translationfeature4567 = pd.Series(translationseries.resample('2s').apply(c_f_p_r))  #calculating feature 4 5 6 7 


# computing feature extraction for swipe
rotatephase = [i for i in rotatedf['ImpinjPhase']]
rotatechannel = [i for i in rotatedf['ChannelIndex']]
rotatetuple = list(zip(rotatephase,rotatechannel))
    
rotateseries = pd.Series(rotatetuple, index=rotatedf['datetime']) #tuple is formatted(phase,channel)

rotatefeature4567 = pd.Series(rotateseries.resample('2s').apply(c_f_p_r))  #calculating feature 4 5 6 7 




[-4.0, 19204, 887.2804327922712, 2621.5949343863176] 67
[0.0, 10764, 641.3374345210427, 2688.0310547271865] 72
[0.0, 31068, 968.2238528997146, 1127.2026338586056] 78
[0.0, 40632, 1255.1510823926085, 2504.522629165087] 74
[0.0, 6560, 449.1724289933009, 3052.523939300067] 74
[0.0, 32800, 1012.7608941780097, 2603.378864305215] 74
[0.0, 18956, 768.7691861270596, 2797.2975530229323] 76
[0.0, 41336, 1145.2621903369484, 2180.9553054669514] 76
[0.0, 34488, 1223.007406103747, 2166.2594489118796] 75
[0.0, 8628, 509.85053251042655, 2987.779108301014] 77
[2.0, 27492, 931.8484765954888, 2673.8631228991508] 75
[0.0, 4860, 363.4954448979223, 3063.6583073483193] 76
[4.0, 25076, 886.899972679254, 1924.6398104580503] 76
[0.0, 20848, 945.5127541701214, 2012.497729467318] 76
[0.0, 10264, 551.609425703863, 2855.3642616427533] 74
[0.0, 34080, 1027.5100298967702, 2524.795472464607] 76
[0.0, 4608, 365.7529741440773, 2827.6484187507936] 75
[0.0, 22996, 844.7782913513234, 1997.6790533016058] 76
[0.0, 35192, 116

[0.0, 22752, 841.6475006703314, 2391.3763958579725] 76
[0.0, 31376, 1116.5654712612707, 652.6582907191515] 74
[0.0, 23400, 965.1010921311752, 2548.655732821607] 72
[2.0, 14512, 631.73912374125, 2421.3794415580555] 73
[0.0, 47408, 1569.9038221780634, 2530.255426896001] 69
[0.0, 31056, 990.3690532011092, 1923.6043714282252] 75
[0.0, 38704, 1134.0507330119538, 1766.6642012561413] 73
[0.0, 31428, 1182.1946678449879, 2718.3127119593873] 73
[0.0, 21684, 794.1620756700582, 2789.8864970938635] 73
[0.0, 39052, 1402.5552268026538, 2548.191792828959] 70
[0.0, 27164, 944.914945361885, 1733.1443999595392] 74
[0.0, 33848, 1101.277915453245, 1880.341576534552] 72
[0.0, 32776, 1185.3134589228866, 2303.3634344391057] 73
[0.0, 14808, 651.4787999068815, 2834.2063282533104] 74
[0.0, 39436, 1490.5652460647786, 2296.1483154749253] 70
[0.0, 16692, 728.0653833145429, 1712.2725120597936] 75
[0.0, 23560, 887.0137793569731, 1728.4572440313484] 69
[0.0, 47284, 1394.553756308796, 2153.276118950945] 72
[4.0, 20360,

# Removing Nan values / merging both dataframes

In [ ]:





#clearning rows with missing rows of data
stillfeature1 = stillfeature1.dropna()
stillfeature2 = stillfeature2.dropna()
stillfeature3 = stillfeature3.dropna()
stillfeature4567 = stillfeature4567.dropna()
stillfeature8 = stillfeature8.dropna()

translationfeature1 = translationfeature1.dropna()
translationfeature2 = translationfeature2.dropna()
translationfeature3 = translationfeature3.dropna()
translationfeature4567 = translationfeature4567.dropna()
translationfeature8 = translationfeature8.dropna()


# formatting data where the 2 data frames contain the same number of rows since
stillfeature1 = stillfeature1[:175]
stillfeature2 = stillfeature2[:175]
stillfeature3 = stillfeature3[:175]
stillfeature4567 = stillfeature4567[:175]
stillfeature8 = stillfeature8[:175]


translationfeature1 = translationfeature1[:175]
translationfeature2 = translationfeature2[:175]
translationfeature3 = translationfeature3[:175]
translationfeature4567 = translationfeature4567[:175]
translationfeature8 = translationfeature8[:175]




newstilldf = pd.DataFrame({
                      'feature1': stillfeature1.values,
                      'feature2':stillfeature2.values,
                      'feature3': stillfeature3.values,
                      'feature4' : [i[0] for i in stillfeature4567],
                      'feature5' : [i[1] for i in stillfeature4567],
                      'feature6' : [i[2] for i in stillfeature4567],
                     'feature7' : [i[3] for i in stillfeature4567]
                      #'feature8': stillfeature8.values
                     })



newtranslationdf = pd.DataFrame({
                    'feature1': translationfeature1.values,
                    'feature2': translationfeature2.values,
                    'feature3': translationfeature3.values,
                    'feature4' : [i[0] for i in translationfeature4567],
                     'feature5' : [i[1] for i in translationfeature4567],
                      'feature6' : [i[2] for i in translationfeature4567],
                     'feature7' : [i[3] for i in translationfeature4567]
                      #'feature8': stillfeature8.values
                     })

# merging both Dataframes into one which will be fed into the SVM Classifier
mergedDF = pd.concat([newstilldf,  newtranslationdf], ignore_index=True)



print(newstilldf)

print('***********')

print(newtranslationdf)

    

# Running the Classifier 

In [ ]:

classes = [] # creating labels for the ground truth
for x in range(mergedDF.shape[0]):
    if x >= mergedDF.shape[0] // 2:
        classes.append(0)
    else:
        classes.append(1)
        

        

X_train, X_test, y_train, y_test = train_test_split(mergedDF, classes, test_size = 0.05) 
svclassifier = SVC(kernel='linear',verbose=1)  
svclassifier.fit(X_train, y_train)  


In [ ]:
y_pred = svclassifier.predict(X_test)  
print(confusion_matrix(y_test,y_pred))  
print(accuracy_score(y_test,y_pred)) 

# Getting Accuracy and weight values

In [ ]:
print(svclassifier._get_coef())
print(svclassifier.score(X_test,y_test))